In [30]:
import keras
import pandas as pd
from keras.layers import Dense, Activation, Embedding, Input, Concatenate, Flatten
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
import progressbar

In [17]:
test = pd.read_csv('../New_Data/embedded_test.csv')
train = pd.read_csv('../New_Data/embedded_data.csv')

In [18]:
#Case 1: User missing in training but song is present (in training)
msno_song = test[~test.msno.isin(train.msno) & test.song_id.isin(train.song_id)]

#Case 2: song missing in training but User exists
song_msno = test[~test.song_id.isin(train.song_id) & test.msno.isin(train.msno)]

#Case 3: both user and song is new
both_missing = test[~test.msno.isin(train.msno) & ~test.song_id.isin(train.song_id)]

In [21]:
no_user_model = keras.models.load_model('../Models/dnn_no_user.h5', compile = False)
no_user_model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

no_song_model = keras.models.load_model('../Models/dnn_no_song.h5', compile = False)
no_song_model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

no_song_no_user_model = keras.models.load_model('../Models/dnn_no_user_no_song.h5', compile = False)
no_song_no_user_model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
batch_size = 32768
lst = []
# Case 1 testing
predicted = no_user_model.predict([msno_song.song_id, msno_song.source_system_tab, msno_song.source_screen_name, msno_song.source_type], batch_size=batch_size, verbose=2)

for x in range(len(predicted)):
    lst.append(predicted[x][1])

In [26]:
predicted = no_song_model.predict([song_msno.msno, song_msno.source_system_tab, song_msno.source_screen_name, song_msno.source_type], batch_size=batch_size, verbose=2)

for x in range(len(predicted)):
    lst.append(predicted[x][1])

In [27]:
predicted = no_song_no_user_model.predict([both_missing.source_system_tab, both_missing.source_screen_name, both_missing.source_type], batch_size=batch_size, verbose=2)

for x in range(len(predicted)):
    lst.append(predicted[x][1])

In [29]:
Y_list = []
Y_list.extend(both_missing.id)
Y_list.extend(song_msno.id)
Y_list.extend(msno_song.id)
headers = ['id', 'target']
new_test = pd.DataFrame(columns=headers)
new_test['id'] = Y_list
new_test['target'] = lst
new_test.to_csv('../Test/cold_start_dnn.csv', index=False, header=headers)

In [31]:
pravar = pd.read_csv('../Test/submission.csv')
piyush = pd.read_csv('../Test/submission_30.csv')
subset = pravar[pravar.id.isin(new_test.id)]
subset = subset.merge(piyush, on='id', how='left')
subset = subset.merge(new_test, on='id', how='left')
subset['target'] = (subset['target_x'] + subset['target_y'] + subset.target)/3

In [ ]:
bar = progressbar.ProgressBar()
subset_ids = set(new_test.id.unique())
for rownum in bar(range(len(pravar))):
    id_ = pravar.iloc[rownum]['id']
    if id_ in subset_ids:
        pravar.set_value(rownum, 'target', subset[subset.id == id_].target)

In [ ]:
headers = ['id', 'target']
pravar.to_csv('../Test/submission_cold_dnn.csv', index=False, header=headers)